In [1]:
# -*- coding: utf-8 -*-
"""
서울교통공사 지연증명서 스크레이퍼 (최종 버전)
- 페이지: http://www.seoulmetro.co.kr/kr/delayProofList.do?

기능 요약
1) select#view_date 의 모든 유효 날짜 옵션을 순회합니다.
2) 각 옵션을 선택한 뒤 옆의 '검색' 버튼(ajax)을 클릭해 결과를 갱신합니다.
3) 결과 테이블(tbl-type1)을 BeautifulSoup으로 파싱하면서 rowspan/colspan을 실제 셀로 전개합니다.
4) 헤더가 다소 변형되어도 정규식으로 [노선, 첫차~09시, 09시~18시, 18시~막차]를 매핑합니다.
5) 최종 CSV는 고정 5컬럼 [날짜, 노선, 첫차~09시, 09시~18시, 18시~막차]만 포함합니다.
6) 날짜 형변환(datetime)은 하지 않습니다(문자열 유지).
7) 결과가 비어도 해당 날짜에 대해 한 줄을 남깁니다(노선/시간대 공란).

사전 준비
- pip install selenium webdriver-manager pandas beautifulsoup4
- Chrome 설치 (webdriver-manager가 자동으로 드라이버를 맞춰 설치)

주의
- 사이트 구조가 변경되면 CSS 셀렉터를 조정하세요.
"""

import re
import time
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# =========================================
# Selenium 초기화
# =========================================
def build_driver(headless: bool = True, detach: bool = False) -> webdriver.Chrome:
    """
    Chrome WebDriver 초기화.
    - headless=True: 창 없이 실행(배치/서버 환경)
    - detach=True: 디버깅 시 창 유지
    """
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1400,2200")
    if detach:
        options.add_experimental_option("detach", True)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_page_load_timeout(60)
    driver.implicitly_wait(2)  # 기본 짧은 암묵적 대기(명시적 대기와 병행)
    return driver


# =========================================
# 공통 유틸: 테이블 접근/대기
# =========================================
def safe_get_table(driver):
    """현재 DOM에서 tbl-type1 테이블 WebElement를 찾아 반환. 없으면 None."""
    try:
        return driver.find_element(By.CSS_SELECTOR, "table.tbl-type1")
    except Exception:
        return None


def table_signature(tbl) -> str:
    """현재 테이블의 간단한 시그니처(행수:텍스트길이). 갱신 감지에 사용."""
    if not tbl:
        return "None"
    try:
        tbody = tbl.find_element(By.CSS_SELECTOR, "tbody")
        rows = tbody.find_elements(By.CSS_SELECTOR, "tr")
        return f"{len(rows)}:{len(tbody.text)}"
    except Exception:
        return "Err"


def wait_for_update_or_timeout(driver, prev_signature: str, timeout: int = 15) -> None:
    """
    ajax 갱신을 기다리되, 타임아웃이 나면 예외 없이 진행합니다.
    테이블(tbody) 시그니처(행수+텍스트길이)가 바뀌면 갱신으로 간주.
    """
    t_end = time.time() + timeout
    while time.time() < t_end:
        tbl = safe_get_table(driver)
        if tbl:
            try:
                tbody = tbl.find_element(By.CSS_SELECTOR, "tbody")
                rows = tbody.find_elements(By.CSS_SELECTOR, "tr")
                sig = f"{len(rows)}:{len(tbody.text)}"
                if sig != prev_signature:
                    return  # 변화 감지 → 대기 종료
            except Exception:
                pass
        time.sleep(0.2)
    # 타임아웃이어도 계속 진행(빈 결과여도 스크랩해야 하므로)


# =========================================
# 테이블 파싱 (rowspan/colspan 전개)
# =========================================
def _clean_text(s: str) -> str:
    """셀 텍스트 전처리: 연속 공백/줄바꿈 정리, NBSP→space."""
    s = (s or "").replace("\xa0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s


def _build_grid(tr_nodes) -> list[list[str]]:
    """
    주어진 <tr> 노드 리스트에서 td/th를 읽어 rowspan/colspan을 전개해
    2차원 리스트(grid)로 반환.
    """
    grid = []
    # col_index -> (value, remaining_rows)
    carry = {}

    for tr in tr_nodes:
        row = []
        col_idx = 0

        def fill_spans():
            nonlocal col_idx
            while col_idx in carry:
                val, remain = carry[col_idx]
                row.append(val)
                remain -= 1
                if remain <= 0:
                    del carry[col_idx]
                else:
                    carry[col_idx] = (val, remain)
                col_idx += 1

        # 시작 위치에서 기존 rowspan 채우기
        fill_spans()

        # 현재 행의 셀들 처리(td/th 모두 허용)
        cells = tr.find_all(["td", "th"], recursive=False)
        for cell in cells:
            text = _clean_text(cell.get_text(separator=" "))
            cs = int(cell.get("colspan", 1) or 1)
            rs = int(cell.get("rowspan", 1) or 1)

            # carry가 있는 위치는 먼저 채움
            fill_spans()

            # colspan 만큼 같은 값을 옆으로 복제
            for _ in range(cs):
                row.append(text)
                # rowspan이 있다면 아래 행들에도 동일 값 채우기 예약
                if rs > 1:
                    carry[col_idx] = (text, rs - 1)
                col_idx += 1

        # 행 끝에서 뒤쪽 연속 carry가 있으면 이어서 채움
        fill_spans()

        grid.append(row)

    # 행마다 길이를 최대 길이에 맞춰 패딩
    max_w = max((len(r) for r in grid), default=0)
    grid = [r + [""] * (max_w - len(r)) for r in grid]
    return grid


def _dedupe_headers(headers):
    """
    중복 헤더명을 유일하게 만들어 DataFrame 인덱싱 충돌을 줄입니다.
    예: ['구분', '구분', '09시~18시'] → ['구분', '구분.2', '09시~18시']
    """
    seen = {}
    new = []
    for h in headers:
        cnt = seen.get(h, 0)
        if cnt == 0:
            new.append(h)
        else:
            new.append(f"{h}.{cnt+1}")
        seen[h] = cnt + 1
    return new


def parse_table_to_df_rowspan(table_element) -> pd.DataFrame:
    """
    tbl-type1을 BeautifulSoup으로 파싱해 thead/tbody를 각각 rowspan/colspan 전개.
    """
    if table_element is None:
        return pd.DataFrame()

    html = table_element.get_attribute("outerHTML")
    soup = BeautifulSoup(html, "html.parser")

    # 1) 헤더 그리드 (thead 우선, 없으면 빈 리스트)
    thead_trs = soup.select("thead tr")
    header_grid = _build_grid(thead_trs) if thead_trs else []

    # 헤더 레이블 생성: 여러 행이면 같은 열의 텍스트를 공백으로 결합
    headers = []
    if header_grid:
        W = len(header_grid[0])
        for c in range(W):
            parts = []
            for r in range(len(header_grid)):
                txt = _clean_text(header_grid[r][c])
                if txt:
                    parts.append(txt)
            headers.append(" ".join(parts).strip())

    # 2) 본문 그리드 (tbody의 모든 tr)
    tbody_trs = soup.select("tbody tr")
    body_grid = _build_grid(tbody_trs)

    if not body_grid:
        return pd.DataFrame()

    # 3) 헤더 길이/본문 열 수 보정 및 중복 헤더 해소
    body_w = max((len(r) for r in body_grid), default=0)
    if not headers or len(headers) != body_w:
        headers = headers + [f"col_{i+1}" for i in range(len(headers), body_w)]
    headers = _dedupe_headers(headers)

    # 4) DataFrame 구성 + 완전 빈 행 제거
    df = pd.DataFrame(body_grid, columns=headers)
    if not df.empty:
        df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)

    return df


# =========================================
# 스키마 정규화 (고정 5컬럼)
# =========================================
TARGET_COLS = ["날짜", "노선", "첫차~09시", "09시~18시", "18시~막차"]

def _canon(s: str) -> str:
    """헤더 비교용 정규화: 공백/콜론 제거 등."""
    s = (s or "").strip()
    s = s.replace("\xa0", " ")
    s = re.sub(r"\s+", "", s)
    s = s.replace(":", "").replace("：", "")
    return s

# 헤더 매칭을 위한 느슨한 정규식들(한글시/콜론/공백 변형 허용)
PAT_LINE     = re.compile(r"(노선|구분)", re.I)           # '구분'을 노선으로 쓰는 경우가 많음
PAT_1_9      = re.compile(r"(첫차).*?0?9시?", re.I)       # '첫차~09시', '첫차~09:00' 등
PAT_9_18     = re.compile(r"0?9시?.*?1?8시?", re.I)       # '09시~18시'
PAT_18_LAST  = re.compile(r"1?8시?.*?(막차|말차)", re.I)  # '18시~막차' (오타 '말차' 방어)


def _first_nonempty_across_cols(df_like: pd.DataFrame) -> pd.Series:
    """
    같은 이름(중복 헤더)으로 여러 열이 묶여 DataFrame이 들어온 경우,
    각 행에서 '첫 번째로 비어있지 않은 문자열'을 골라 Series로 축약합니다.
    """
    tmp = df_like.astype(str).applymap(lambda x: x.strip())
    return tmp.apply(lambda row: next((v for v in row if v), ""), axis=1)


def _extract_series(df_raw: pd.DataFrame, src):
    """
    df_raw[src]가 Series면 문자열 strip 해서 반환,
    DataFrame이면 행 단위로 축약해 Series로 반환(중복 헤더 안전).
    """
    col_obj = df_raw[src]
    if isinstance(col_obj, pd.DataFrame):
        return _first_nonempty_across_cols(col_obj)
    else:
        return col_obj.astype(str).str.strip()


def normalize_delay_table(df_raw: pd.DataFrame, date_label: str) -> pd.DataFrame:
    """
    임의 헤더 df_raw → 고정 스키마 [날짜, 노선, 첫차~09시, 09시~18시, 18시~막차]
    - rowspan/colspan 전개된 df_raw를 대상으로 동작
    - 헤더 매칭 실패 시 최선의 추정으로 채움
    - 중복 헤더도 안전 처리
    """
    if df_raw is None or df_raw.empty:
        return pd.DataFrame([{
            "날짜": date_label, "노선": "", "첫차~09시": "", "09시~18시": "", "18시~막차": ""
        }], columns=TARGET_COLS)

    # 1) 헤더 매핑
    colmap = {"노선": None, "첫차~09시": None, "09시~18시": None, "18시~막차": None}
    for c in df_raw.columns:
        c_can = _canon(str(c))
        if colmap["노선"] is None and (PAT_LINE.search(c_can) or c_can in {"호선", "노선"}):
            colmap["노선"] = c
            continue
        if colmap["첫차~09시"] is None and PAT_1_9.search(c_can):
            colmap["첫차~09시"] = c
            continue
        if colmap["09시~18시"] is None and PAT_9_18.search(c_can):
            colmap["09시~18시"] = c
            continue
        if colmap["18시~막차"] is None and PAT_18_LAST.search(c_can):
            colmap["18시~막차"] = c
            continue

    # 2) '노선'을 못 찾으면 첫 컬럼으로 추정
    if colmap["노선"] is None and len(df_raw.columns) > 0:
        colmap["노선"] = df_raw.columns[0]

    # 3) 스키마 구성 (날짜는 형변환 없이 원문 문자열 유지)
    out = pd.DataFrame()
    out["날짜"] = [date_label] * len(df_raw)

    for tgt in ["노선", "첫차~09시", "09시~18시", "18시~막차"]:
        src = colmap.get(tgt)
        if src is None or src not in df_raw.columns:
            out[tgt] = ""
        else:
            out[tgt] = _extract_series(df_raw, src)

    # 완전 빈 DF 방어
    if out.empty:
        out = pd.DataFrame([{
            "날짜": date_label, "노선": "", "첫차~09시": "", "09시~18시": "", "18시~막차": ""
        }], columns=TARGET_COLS)

    # 컬럼 순서 고정
    out = out[TARGET_COLS]
    return out


# =========================================
# 메인 파이프라인
# =========================================
def scrape_delay_proofs(
    url: str = "http://www.seoulmetro.co.kr/kr/delayProofList.do?",
    output_csv: str = "delay_proof_all.csv",
    headless: bool = True
):
    driver = build_driver(headless=headless)
    all_dfs = []

    # 페이지의 실제 구조에 맞게 필요 시 수정
    SELECT_LOC = (By.CSS_SELECTOR, "select#view_date")
    SEARCH_LOC = (By.CSS_SELECTOR, "a[href^='javascript:document.searchForm.submit']")

    try:
        driver.get(url)
        time.sleep(1.0)  # 초기 렌더 안정화

        WebDriverWait(driver, 15).until(EC.presence_of_element_located(SELECT_LOC))
        WebDriverWait(driver, 15).until(EC.element_to_be_clickable(SEARCH_LOC))

        # 1) 날짜 옵션 수집 (placeholder/빈 값 제외)
        select_elem = driver.find_element(*SELECT_LOC)
        select_obj = Select(select_elem)
        option_meta = []
        for opt in select_obj.options:
            text = (opt.text or "").strip()
            value = (opt.get_attribute("value") or "").strip()
            if not value or "선택" in text:  # 예: "날짜선택" 같은 placeholder
                continue
            option_meta.append({"text": text, "value": value})

        if not option_meta:
            raise RuntimeError("select#view_date 에 유효한 날짜 option이 없습니다.")

        print(f"감지된 날짜 옵션 수: {len(option_meta)}")

        # 초기 테이블 시그니처
        prev_sig = table_signature(safe_get_table(driver))

        # 2) 각 날짜 옵션 순회
        for idx, meta in enumerate(option_meta, start=1):
            date_label = meta["text"] or meta["value"]

            # 매 회전마다 fresh 조회(리렌더 대비)
            select_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located(SELECT_LOC))
            select_obj  = Select(select_elem)

            # value 우선, 실패 시 visible_text
            try:
                select_obj.select_by_value(meta["value"])
            except Exception:
                select_obj.select_by_visible_text(meta["text"])

            # 검색 버튼 클릭(JS로 안정 처리)
            search_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(SEARCH_LOC))
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", search_btn)
            time.sleep(0.05)
            driver.execute_script("arguments[0].click();", search_btn)

            # ajax 갱신(또는 타임아웃) 대기 → 타임아웃이어도 계속
            wait_for_update_or_timeout(driver, prev_signature=prev_sig, timeout=20)
            prev_sig = table_signature(safe_get_table(driver))
            time.sleep(0.15)  # 아주 짧은 안정화

            # 3) 파싱 (rowspan/colspan 전개)
            table_el = safe_get_table(driver)
            raw_df = parse_table_to_df_rowspan(table_el)

            # 4) 스키마 정규화(고정 5컬럼), 빈 결과면 한 줄 생성
            norm_df = normalize_delay_table(raw_df, date_label=date_label)
            all_dfs.append(norm_df)

            print(f"[진행] {idx}/{len(option_meta)}: {date_label} → 행수={len(norm_df)}")

        # 5) 전체 병합/저장 (날짜 형변환 없음)
        final_df = pd.concat(all_dfs, ignore_index=True)
        final_df = final_df[["날짜", "노선", "첫차~09시", "09시~18시", "18시~막차"]]
        final_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
        print(f"[완료] CSV 저장: {output_csv} (총 행수={len(final_df)})")

    finally:
        try:
            driver.quit()
        except Exception:
            pass


if __name__ == "__main__":
    # 디버깅 시 headless=False로 변경해 실제 DOM 변화를 눈으로 확인해 보세요.
    scrape_delay_proofs(
        url="http://www.seoulmetro.co.kr/kr/delayProofList.do?",
        output_csv="delay_proof_all.csv",
        headless=True
    )


감지된 날짜 옵션 수: 31


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_21212\1311062769.py:231: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)


[진행] 1/31: 금일 (2025-09-16) → 행수=23


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_21212\1311062769.py:231: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)


[진행] 2/31: 1일전 (2025-09-15) → 행수=23


...

[진행] 29/31: 28일전 (2025-08-19) → 행수=23


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_21212\1311062769.py:231: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)


[진행] 30/31: 29일전 (2025-08-18) → 행수=23


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_21212\1311062769.py:231: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)


[진행] 31/31: 30일전 (2025-08-17) → 행수=23
[완료] CSV 저장: delay_proof_all.csv (총 행수=713)


In [2]:
# -*- coding: utf-8 -*-
"""
서울교통공사 지연증명서 스크레이퍼 (방향/순서 최종 조정)
- 페이지: http://www.seoulmetro.co.kr/kr/delayProofList.do?

핵심 기능
1) select#view_date 의 모든 날짜 옵션을 순회
2) '검색' 클릭(ajax) 후 결과 테이블(tbl-type1) 로딩
3) BeautifulSoup으로 thead/tbody를 파싱하며 rowspan/colspan 전개
4) 헤더/값 휴리스틱으로 [노선, 방향(상/하행·내/외선), 3개 시간대] 매핑
5) 최종 CSV 스키마 고정: [날짜, 노선, 방향, 첫차~09시, 09시~18시, 18시~막차]
6) 결과가 비어도 해당 날짜 한 줄 생성(노선/방향/시간대는 공란)
7) 날짜 형변환(datetime) 미적용(문자열 유지)

사전 준비:
pip install selenium webdriver-manager pandas beautifulsoup4
"""

import re
import time
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# ==========================
# Selenium 초기화
# ==========================
def build_driver(headless: bool = True, detach: bool = False) -> webdriver.Chrome:
    """Chrome WebDriver 초기화."""
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1400,2200")
    if detach:
        options.add_experimental_option("detach", True)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_page_load_timeout(60)
    driver.implicitly_wait(2)
    return driver


# ==========================
# 공통 유틸: 테이블 접근/대기
# ==========================
def safe_get_table(driver):
    """현재 DOM에서 tbl-type1 테이블 요소를 찾아 반환. 없으면 None."""
    try:
        return driver.find_element(By.CSS_SELECTOR, "table.tbl-type1")
    except Exception:
        return None


def table_signature(tbl) -> str:
    """현재 테이블의 간단한 시그니처(행수:텍스트길이)."""
    if not tbl:
        return "None"
    try:
        tbody = tbl.find_element(By.CSS_SELECTOR, "tbody")
        rows = tbody.find_elements(By.CSS_SELECTOR, "tr")
        return f"{len(rows)}:{len(tbody.text)}"
    except Exception:
        return "Err"


def wait_for_update_or_timeout(driver, prev_signature: str, timeout: int = 15) -> None:
    """
    ajax 갱신을 기다리되, 타임아웃이면 예외 없이 진행.
    테이블(tbody) 시그니처(행수+텍스트길이)가 바뀌면 갱신으로 간주.
    """
    t_end = time.time() + timeout
    while time.time() < t_end:
        tbl = safe_get_table(driver)
        if tbl:
            try:
                tbody = tbl.find_element(By.CSS_SELECTOR, "tbody")
                rows = tbody.find_elements(By.CSS_SELECTOR, "tr")
                sig = f"{len(rows)}:{len(tbody.text)}"
                if sig != prev_signature:
                    return
            except Exception:
                pass
        time.sleep(0.2)
    # 타임아웃이어도 계속 진행(빈 결과도 기록해야 하므로)


# ==========================
# 테이블 파싱 (rowspan/colspan 전개)
# ==========================
def _clean_text(s: str) -> str:
    """셀 텍스트 전처리: 연속 공백/줄바꿈 정리, NBSP→space."""
    s = (s or "").replace("\xa0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s


def _build_grid(tr_nodes) -> list[list[str]]:
    """
    주어진 <tr> 리스트에서 td/th를 읽어 rowspan/colspan을 전개해 2차원 리스트(grid)로 반환.
    """
    grid = []
    # col_index -> (value, remaining_rows)
    carry = {}

    for tr in tr_nodes:
        row = []
        col_idx = 0

        def fill_spans():
            nonlocal col_idx
            while col_idx in carry:
                val, remain = carry[col_idx]
                row.append(val)
                remain -= 1
                if remain <= 0:
                    del carry[col_idx]
                else:
                    carry[col_idx] = (val, remain)
                col_idx += 1

        fill_spans()  # 기존 rowspan 채우기

        cells = tr.find_all(["td", "th"], recursive=False)
        for cell in cells:
            text = _clean_text(cell.get_text(separator=" "))
            cs = int(cell.get("colspan", 1) or 1)
            rs = int(cell.get("rowspan", 1) or 1)

            fill_spans()

            for _ in range(cs):  # colspan 만큼 옆으로 복제
                row.append(text)
                if rs > 1:       # 아래 행들 예약(rowspan)
                    carry[col_idx] = (text, rs - 1)
                col_idx += 1

        fill_spans()
        grid.append(row)

    max_w = max((len(r) for r in grid), default=0)
    grid = [r + [""] * (max_w - len(r)) for r in grid]
    return grid


def _dedupe_headers(headers):
    """중복 헤더명을 유일하게 만들어 인덱싱 충돌을 줄임."""
    seen = {}
    new = []
    for h in headers:
        cnt = seen.get(h, 0)
        if cnt == 0:
            new.append(h)
        else:
            new.append(f"{h}.{cnt+1}")
        seen[h] = cnt + 1
    return new


def parse_table_to_df_rowspan(table_element) -> pd.DataFrame:
    """tbl-type1을 BeautifulSoup으로 파싱해 thead/tbody를 각각 rowspan/colspan 전개."""
    if table_element is None:
        return pd.DataFrame()

    html = table_element.get_attribute("outerHTML")
    soup = BeautifulSoup(html, "html.parser")

    # 헤더
    thead_trs = soup.select("thead tr")
    header_grid = _build_grid(thead_trs) if thead_trs else []
    headers = []
    if header_grid:
        W = len(header_grid[0])
        for c in range(W):
            parts = []
            for r in range(len(header_grid)):
                txt = _clean_text(header_grid[r][c])
                if txt:
                    parts.append(txt)
            headers.append(" ".join(parts).strip())

    # 본문
    tbody_trs = soup.select("tbody tr")
    body_grid = _build_grid(tbody_trs)
    if not body_grid:
        return pd.DataFrame()

    body_w = max((len(r) for r in body_grid), default=0)
    if not headers or len(headers) != body_w:
        headers = headers + [f"col_{i+1}" for i in range(len(headers), body_w)]
    headers = _dedupe_headers(headers)

    df = pd.DataFrame(body_grid, columns=headers)
    if not df.empty:
        df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)
    return df


# ==========================
# 스키마 정규화 (고정 6컬럼)
# ==========================
TARGET_COLS = ["날짜", "노선", "방향", "첫차~09시", "09시~18시", "18시~막차"]

def _canon(s: str) -> str:
    """헤더 비교용 정규화: 공백/콜론 제거 등."""
    s = (s or "").strip()
    s = s.replace("\xa0", " ")
    s = re.sub(r"\s+", "", s)
    s = s.replace(":", "").replace("：", "")
    return s

# 헤더 패턴(느슨)
PAT_TYPE_HDR   = re.compile(r"(방향|유형|방향|운행방향|열차방향|방향구분|발급구분)", re.I)
PAT_LINE_HDR   = re.compile(r"(노선|호선)", re.I)     # 명시적 노선/호선
PAT_AMBIG_HDR  = re.compile(r"(구분)", re.I)          # '구분'은 상황에 따라 노선/방향

# 시간대
PAT_1_9        = re.compile(r"(첫차).*?0?9시?", re.I)
PAT_9_18       = re.compile(r"0?9시?.*?1?8시?", re.I)
PAT_18_LAST    = re.compile(r"1?8시?.*?(막차|말차)", re.I)

# 값 패턴: 노선과 방향
VAL_LINE_RE       = re.compile(r"(?:\d+\s*호선|[가-힣A-Za-z]+선)$")
VAL_DIRECTION_RE  = re.compile(r"(상행선?|하행선?|상선|하선|외선|내선)$")

def _first_nonempty_across_cols(df_like: pd.DataFrame) -> pd.Series:
    """중복 헤더로 DataFrame이 들어오면 행별 첫 비어있지 않은 값을 선택해 Series로 축약."""
    tmp = df_like.astype(str).applymap(lambda x: x.strip())
    return tmp.apply(lambda row: next((v for v in row if v), ""), axis=1)

def _extract_series(df_raw: pd.DataFrame, src):
    """df_raw[src]가 Series/DF 모두 안전하게 문자열 Series로 변환."""
    col_obj = df_raw[src]
    if isinstance(col_obj, pd.DataFrame):
        return _first_nonempty_across_cols(col_obj)
    else:
        return col_obj.astype(str).str.strip()

def _looks_like_line(series: pd.Series) -> bool:
    """값 분포가 노선 형태(…호선/…선)에 가깝다면 True."""
    vals = series.astype(str).str.strip()
    non_empty = vals[vals != ""]
    if len(non_empty) == 0:
        return False
    m = (non_empty.str.contains(VAL_LINE_RE)).mean()
    return m >= 0.5

def _looks_like_direction(series: pd.Series) -> bool:
    """값 분포가 방향(상행·하행·내선·외선)에 가깝다면 True."""
    vals = series.astype(str).str.strip()
    non_empty = vals[vals != ""]
    if len(non_empty) == 0:
        return False
    # '상행', '하행'처럼 '선'이 빠진 값도 허용
    m = (non_empty.str.contains(VAL_DIRECTION_RE) | non_empty.str.contains(r"(상행|하행)$")).mean()
    return m >= 0.5

def _split_line_and_dir(cell: str) -> tuple[str, str]:
    """
    '1호선 상행선'·'수인분당선 내선' 같은 한 셀을 (노선, 방향)로 분리.
    매칭 실패 시 (원문, '') 반환.
    """
    txt = _clean_text(cell)
    if not txt:
        return "", ""
    m = re.search(r"(상행선?|하행선?|상선|하선|외선|내선)$", txt)
    if not m:
        # '... 상행' 처럼 '선' 없는 표기도 보정
        m = re.search(r"(상행|하행)$", txt)
    if m:
        dir_raw = m.group(0)
        # 표기 표준화: '상행' -> '상행선', '상선'은 그대로
        if dir_raw in ("상행", "하행"):
            dir_norm = dir_raw + "선"
        else:
            dir_norm = dir_raw
        line = _clean_text(txt[:m.start()].rstrip())
        return line, dir_norm
    return txt, ""  # 분리 불가 → 원문을 노선으로 유지

def normalize_delay_table(df_raw: pd.DataFrame, date_label: str) -> pd.DataFrame:
    """
    임의 헤더 df_raw → 고정 스키마 [날짜, 노선, 방향, 첫차~09시, 09시~18시, 18시~막차]
    - rowspan/colspan 전개된 df_raw를 대상으로 동작
    - '구분' 컬럼은 값 형태로 노선/방향 중 무엇인지 판단
    - 노선 셀 안에 방향이 함께 적힌 경우 분리
    - 중복 헤더 안전 처리
    """
    if df_raw is None or df_raw.empty:
        return pd.DataFrame([{
            "날짜": date_label, "노선": "", "방향": "", "첫차~09시": "", "09시~18시": "", "18시~막차": ""
        }], columns=TARGET_COLS)

    # 1) 헤더 분류
    explicit_type_cols = []
    explicit_line_cols = []
    ambiguous_cols     = []  # '구분'

    time_1_9 = None
    time_9_18 = None
    time_18_last = None

    for c in df_raw.columns:
        c_can = _canon(str(c))
        if PAT_TYPE_HDR.search(c_can):
            explicit_type_cols.append(c)
            continue
        if PAT_LINE_HDR.search(c_can):
            explicit_line_cols.append(c)
            continue
        if PAT_AMBIG_HDR.search(c_can):
            ambiguous_cols.append(c)
            continue
        if time_1_9 is None and PAT_1_9.search(c_can):
            time_1_9 = c
            continue
        if time_9_18 is None and PAT_9_18.search(c_can):
            time_9_18 = c
            continue
        if time_18_last is None and PAT_18_LAST.search(c_can):
            time_18_last = c
            continue

    # 2) 노선/방향 결정 로직
    chosen_line = explicit_line_cols[0] if explicit_line_cols else None
    chosen_type = explicit_type_cols[0] if explicit_type_cols else None

    # '구분' 컬럼을 값 형태로 분류
    for c in ambiguous_cols:
        series = _extract_series(df_raw, c)
        if chosen_line is None and _looks_like_line(series):
            chosen_line = c
        elif chosen_type is None and _looks_like_direction(series):
            chosen_type = c
        elif chosen_type is None:
            # 그래도 정 못 찾았으면 타입으로 우선 할당 (후보로 남겨둠)
            chosen_type = c

    # 둘 다 비어 있으면 첫 컬럼을 보정
    if chosen_line is None and len(df_raw.columns) > 0:
        probe = df_raw.columns[0]
        ser = _extract_series(df_raw, probe)
        if _looks_like_line(ser):
            chosen_line = probe
        elif _looks_like_direction(ser):
            chosen_type = chosen_type or probe
        else:
            chosen_line = probe  # 최후의 수단

    # 3) 스키마 구성 (날짜는 형변환 없이 문자열)
    out = pd.DataFrame()
    out["날짜"] = [date_label] * len(df_raw)

    ser_line = _extract_series(df_raw, chosen_line) if (chosen_line is not None and chosen_line in df_raw.columns) else pd.Series([""] * len(df_raw))
    ser_type = _extract_series(df_raw, chosen_type) if (chosen_type is not None and chosen_type in df_raw.columns) else pd.Series([""] * len(df_raw))

    # 3-1) 만약 '노선' 셀에 방향이 붙어 있으면 분리(예: '1호선 상행선')
    #      단, 이미 종별(방향) 컬럼이 존재하고 값이 있으면 분리하지 않음.
    if ser_type.str.strip().eq("").all():
        # 행 단위로 분리 시도
        new_line = []
        new_type = []
        for val in ser_line.fillna(""):
            ln, dr = _split_line_and_dir(val)
            new_line.append(ln)
            new_type.append(dr)
        ser_line = pd.Series(new_line)
        # 방향가 비어 있는 행에만 새로 채움
        ser_type = pd.Series(new_type)

    # 3-2) 결과 대입
    out["노선"] = ser_line.fillna("").astype(str).str.strip()
    out["방향"] = ser_type.fillna("").astype(str).str.strip()

    # 3-3) 시간대 3칸
    out["첫차~09시"] = _extract_series(df_raw, time_1_9) if (time_1_9 is not None and time_1_9 in df_raw.columns) else ""
    out["09시~18시"] = _extract_series(df_raw, time_9_18) if (time_9_18 is not None and time_9_18 in df_raw.columns) else ""
    out["18시~막차"] = _extract_series(df_raw, time_18_last) if (time_18_last is not None and time_18_last in df_raw.columns) else ""

    # 완전 빈 DF 방어
    if out.empty:
        out = pd.DataFrame([{
            "날짜": date_label, "노선": "", "방향": "", "첫차~09시": "", "09시~18시": "", "18시~막차": ""
        }], columns=TARGET_COLS)

    # 컬럼 순서 고정
    out = out[TARGET_COLS]
    return out


# ==========================
# 메인 파이프라인
# ==========================
def scrape_delay_proofs(
    url: str = "http://www.seoulmetro.co.kr/kr/delayProofList.do?",
    output_csv: str = "delay_proof_all.csv",
    headless: bool = True
):
    driver = build_driver(headless=headless)
    all_dfs = []

    SELECT_LOC = (By.CSS_SELECTOR, "select#view_date")
    SEARCH_LOC = (By.CSS_SELECTOR, "a[href^='javascript:document.searchForm.submit']")

    try:
        driver.get(url)
        time.sleep(1.0)

        WebDriverWait(driver, 15).until(EC.presence_of_element_located(SELECT_LOC))
        WebDriverWait(driver, 15).until(EC.element_to_be_clickable(SEARCH_LOC))

        # 날짜 옵션 수집 (placeholder 제외)
        select_elem = driver.find_element(*SELECT_LOC)
        select_obj = Select(select_elem)
        option_meta = []
        for opt in select_obj.options:
            text = (opt.text or "").strip()
            value = (opt.get_attribute("value") or "").strip()
            if not value or "선택" in text:
                continue
            option_meta.append({"text": text, "value": value})

        if not option_meta:
            raise RuntimeError("select#view_date 에 유효한 날짜 option이 없습니다.")

        print(f"감지된 날짜 옵션 수: {len(option_meta)}")

        prev_sig = table_signature(safe_get_table(driver))

        for idx, meta in enumerate(option_meta, start=1):
            date_label = meta["text"] or meta["value"]

            # select 재조회(리렌더 대비)
            select_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located(SELECT_LOC))
            select_obj  = Select(select_elem)
            try:
                select_obj.select_by_value(meta["value"])
            except Exception:
                select_obj.select_by_visible_text(meta["text"])

            # 검색 클릭(JS)
            search_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(SEARCH_LOC))
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", search_btn)
            time.sleep(0.05)
            driver.execute_script("arguments[0].click();", search_btn)

            # ajax 갱신(or 타임아웃) 대기
            wait_for_update_or_timeout(driver, prev_signature=prev_sig, timeout=20)
            prev_sig = table_signature(safe_get_table(driver))
            time.sleep(0.15)

            # 파싱 → 정규화(고정 6컬럼, 방향=방향)
            table_el = safe_get_table(driver)
            raw_df = parse_table_to_df_rowspan(table_el)
            norm_df = normalize_delay_table(raw_df, date_label=date_label)
            all_dfs.append(norm_df)

            print(f"[진행] {idx}/{len(option_meta)}: {date_label} → 행수={len(norm_df)}")

        final_df = pd.concat(all_dfs, ignore_index=True)
        final_df = final_df[["날짜", "노선", "방향", "첫차~09시", "09시~18시", "18시~막차"]]
        final_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
        print(f"[완료] CSV 저장: {output_csv} (총 행수={len(final_df)})")

    finally:
        try:
            driver.quit()
        except Exception:
            pass


if __name__ == "__main__":
    # 디버깅 시 headless=False로 실제 DOM 변화를 확인하세요.
    scrape_delay_proofs(
        url="http://www.seoulmetro.co.kr/kr/delayProofList.do?",
        output_csv="delay_proof_all.csv",
        headless=True
    )


감지된 날짜 옵션 수: 31


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\1243344420.py:204: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)


[진행] 1/31: 금일 (2025-09-16) → 행수=23


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\1243344420.py:204: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)


[진행] 2/31: 1일전 (2025-09-15) → 행수=23


...

[진행] 29/31: 28일전 (2025-08-19) → 행수=23


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\1243344420.py:204: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)


[진행] 30/31: 29일전 (2025-08-18) → 행수=23


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\1243344420.py:204: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)


[진행] 31/31: 30일전 (2025-08-17) → 행수=23
[완료] CSV 저장: delay_proof_all.csv (총 행수=713)


In [4]:
# -*- coding: utf-8 -*-
"""
서울교통공사 지연증명서 스크레이퍼 (표 양식 충실 + 병합셀 전개 + 방향 자동매핑)
- 페이지: http://www.seoulmetro.co.kr/kr/delayProofList.do?

주요 기능
1) select#view_date 의 모든 날짜 옵션을 순회
2) '검색' 클릭(ajax) → 결과 테이블(tbl-type1) 로딩
3) BeautifulSoup으로 thead/tbody를 파싱하며 rowspan/colspan을 실제 셀로 전개(스크린샷 구조 충실)
4) 헤더 유무와 상관없이 값 패턴으로 [노선, 방향(상/하행·내/외선), 3개 시간대]를 매핑
5) '노선' 셀 안에 방향이 붙은 값(예: '1호선 상행선')은 분리
6) 최종 CSV 스키마/순서 고정: [날짜, 노선, 방향, 첫차~09시, 09시~18시, 18시~막차]
7) 결과가 비어도 날짜 한 줄 생성(노선/방향/시간대 공란)
8) 날짜 형변환 미적용(문자열 유지)

의존성:
pip install selenium webdriver-manager pandas beautifulsoup4
"""

import re
import time
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# ==========================
# Selenium 초기화
# ==========================
def build_driver(headless: bool = True, detach: bool = False) -> webdriver.Chrome:
    """Chrome WebDriver 초기화."""
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1400,2200")
    if detach:
        options.add_experimental_option("detach", True)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_page_load_timeout(60)
    driver.implicitly_wait(2)
    return driver


# ==========================
# 공통 유틸: 테이블 접근/대기
# ==========================
def safe_get_table(driver):
    """현재 DOM에서 tbl-type1 테이블 요소를 찾아 반환. 없으면 None."""
    try:
        return driver.find_element(By.CSS_SELECTOR, "table.tbl-type1")
    except Exception:
        return None


def table_signature(tbl) -> str:
    """현재 테이블의 간단한 시그니처(행수:텍스트길이)."""
    if not tbl:
        return "None"
    try:
        tbody = tbl.find_element(By.CSS_SELECTOR, "tbody")
        rows = tbody.find_elements(By.CSS_SELECTOR, "tr")
        return f"{len(rows)}:{len(tbody.text)}"
    except Exception:
        return "Err"


def wait_for_update_or_timeout(driver, prev_signature: str, timeout: int = 15) -> None:
    """
    ajax 갱신을 기다리되, 타임아웃이면 예외 없이 진행.
    테이블(tbody) 시그니처(행수+텍스트길이)가 바뀌면 갱신으로 간주.
    """
    t_end = time.time() + timeout
    while time.time() < t_end:
        tbl = safe_get_table(driver)
        if tbl:
            try:
                tbody = tbl.find_element(By.CSS_SELECTOR, "tbody")
                rows = tbody.find_elements(By.CSS_SELECTOR, "tr")
                sig = f"{len(rows)}:{len(tbody.text)}"
                if sig != prev_signature:
                    return
            except Exception:
                pass
        time.sleep(0.2)
    # 타임아웃이어도 계속 진행(빈 결과도 기록해야 하므로)


# ==========================
# 테이블 파싱 (rowspan/colspan 전개)
# ==========================
def _clean_text(s: str) -> str:
    """셀 텍스트 전처리: 연속 공백/줄바꿈 정리, NBSP→space."""
    s = (s or "").replace("\xa0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s


def _build_grid(tr_nodes) -> list[list[str]]:
    """
    주어진 <tr> 리스트에서 td/th를 읽어 rowspan/colspan을 전개해 2차원 리스트(grid)로 반환.
    - 스크린샷처럼 노선 셀이 rowspan=2, 그 다음 셀에 '상행선/하행선'이 오는 형태를 정확히 보존
    """
    grid = []
    # col_index -> (value, remaining_rows)
    carry = {}

    for tr in tr_nodes:
        row = []
        col_idx = 0

        def fill_spans():
            """위에서 내려오는 rowspan 값들을 현재 행에 먼저 채운다."""
            nonlocal col_idx
            while col_idx in carry:
                val, remain = carry[col_idx]
                row.append(val)
                remain -= 1
                if remain <= 0:
                    del carry[col_idx]
                else:
                    carry[col_idx] = (val, remain)
                col_idx += 1

        fill_spans()  # 기존 rowspan 채우기

        # 현재 행의 셀들 처리(td/th 모두 허용)
        cells = tr.find_all(["td", "th"], recursive=False)
        for cell in cells:
            text = _clean_text(cell.get_text(separator=" "))
            cs = int(cell.get("colspan", 1) or 1)
            rs = int(cell.get("rowspan", 1) or 1)

            fill_spans()

            # colspan 만큼 같은 값을 옆으로 복제
            for _ in range(cs):
                row.append(text)
                # rowspan이 있다면 아래 행들에도 동일 값 채우기 예약
                if rs > 1:
                    carry[col_idx] = (text, rs - 1)
                col_idx += 1

        fill_spans()
        grid.append(row)

    # 행마다 길이를 최대 길이에 맞춰 패딩(열 개수 정렬)
    max_w = max((len(r) for r in grid), default=0)
    grid = [r + [""] * (max_w - len(r)) for r in grid]
    return grid


def _dedupe_headers(headers):
    """중복 헤더명을 유일하게 만들어 인덱싱 충돌을 줄임."""
    seen = {}
    new = []
    for h in headers:
        cnt = seen.get(h, 0)
        if cnt == 0:
            new.append(h)
        else:
            new.append(f"{h}.{cnt+1}")
        seen[h] = cnt + 1
    return new


def parse_table_to_df_rowspan(table_element) -> pd.DataFrame:
    """tbl-type1을 BeautifulSoup으로 파싱해 thead/tbody를 각각 rowspan/colspan 전개."""
    if table_element is None:
        return pd.DataFrame()

    html = table_element.get_attribute("outerHTML")
    soup = BeautifulSoup(html, "html.parser")

    # 1) 헤더 그리드
    thead_trs = soup.select("thead tr")
    header_grid = _build_grid(thead_trs) if thead_trs else []
    headers = []
    if header_grid:
        W = len(header_grid[0])
        for c in range(W):
            parts = []
            for r in range(len(header_grid)):
                txt = _clean_text(header_grid[r][c])
                if txt:
                    parts.append(txt)
            headers.append(" ".join(parts).strip())

    # 2) 본문 그리드
    tbody_trs = soup.select("tbody tr")
    body_grid = _build_grid(tbody_trs)
    if not body_grid:
        return pd.DataFrame()

    # 3) 헤더-본문 폭 보정 및 중복 해소
    body_w = max((len(r) for r in body_grid), default=0)
    if not headers or len(headers) != body_w:
        headers = headers + [f"col_{i+1}" for i in range(len(headers), body_w)]
    headers = _dedupe_headers(headers)

    # 4) DataFrame 구성 + 완전 빈 행 제거
    df = pd.DataFrame(body_grid, columns=headers)
    if not df.empty:
        df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)
    return df


# ==========================
# 스키마 정규화 (고정 6컬럼)
# ==========================
TARGET_COLS = ["날짜", "노선", "방향", "첫차~09시", "09시~18시", "18시~막차"]

def _canon(s: str) -> str:
    """헤더 비교용 정규화: 공백/콜론 제거 등."""
    s = (s or "").strip()
    s = s.replace("\xa0", " ")
    s = re.sub(r"\s+", "", s)
    s = s.replace(":", "").replace("：", "")
    return s

# 헤더 패턴(느슨)
PAT_LINE_HDR   = re.compile(r"(노선|호선)", re.I)     # 명시적 노선/호선
PAT_TYPE_HDR   = re.compile(r"(방향|유형|방향|운행방향|열차방향|방향구분|발급구분)", re.I)
PAT_AMBIG_HDR  = re.compile(r"(구분)", re.I)          # '구분'은 상황에 따라 노선/방향

# 시간대
PAT_1_9        = re.compile(r"(첫차).*?0?9시?", re.I)
PAT_9_18       = re.compile(r"0?9시?.*?1?8시?", re.I)
PAT_18_LAST    = re.compile(r"1?8시?.*?(막차|말차)", re.I)

# 값 패턴: 노선과 방향
VAL_LINE_RE       = re.compile(r"(?:\d+\s*호선|[가-힣A-Za-z]+선)$")
VAL_DIRECTION_RE  = re.compile(r"(상행선?|하행선?|상선|하선|외선|내선)$")

def _first_nonempty_across_cols(df_like: pd.DataFrame) -> pd.Series:
    """중복 헤더로 DataFrame이 들어오면 행별 첫 비어있지 않은 값을 선택해 Series로 축약."""
    tmp = df_like.astype(str).applymap(lambda x: x.strip())
    return tmp.apply(lambda row: next((v for v in row if v), ""), axis=1)

def _extract_series(df_raw: pd.DataFrame, src):
    """df_raw[src]가 Series/DF 모두 안전하게 문자열 Series로 변환."""
    col_obj = df_raw[src]
    if isinstance(col_obj, pd.DataFrame):
        return _first_nonempty_across_cols(col_obj)
    else:
        return col_obj.astype(str).str.strip()

def _looks_like_line(series: pd.Series) -> float:
    """노선 형태(…호선/…선) 비율을 반환."""
    vals = series.astype(str).str.strip()
    non_empty = vals[vals != ""]
    if len(non_empty) == 0:
        return 0.0
    return (non_empty.str.contains(VAL_LINE_RE)).mean()

def _looks_like_direction(series: pd.Series) -> float:
    """방향(상/하행·내/외선) 비율을 반환."""
    vals = series.astype(str).str.strip()
    non_empty = vals[vals != ""]
    if len(non_empty) == 0:
        return 0.0
    return (non_empty.str.contains(VAL_DIRECTION_RE) | non_empty.str.contains(r"(상행|하행)$")).mean()

def _split_line_and_dir(cell: str) -> tuple[str, str]:
    """
    '1호선 상행선'·'수인분당선 내선' 같은 한 셀을 (노선, 방향)로 분리.
    매칭 실패 시 (원문, '') 반환.
    """
    txt = _clean_text(cell)
    if not txt:
        return "", ""
    m = re.search(r"(상행선?|하행선?|상선|하선|외선|내선)$", txt)
    if not m:
        m = re.search(r"(상행|하행)$", txt)  # '선' 생략 표기도 허용
    if m:
        dir_raw = m.group(0)
        dir_norm = (dir_raw + "선") if dir_raw in ("상행", "하행") else dir_raw
        line = _clean_text(txt[:m.start()].rstrip())
        return line, dir_norm
    return txt, ""


def _choose_direction_column(df_raw: pd.DataFrame, exclude_cols: set) -> str | None:
    """
    헤더가 없더라도 값 패턴으로 '방향(방향)' 컬럼을 선택.
    - 시간대/노선으로 이미 매핑된 컬럼은 제외
    - 방향값 비율이 가장 높은 열을 선택(임계 0.3)
    """
    best_c, best_score = None, 0.0
    for c in df_raw.columns:
        if c in exclude_cols:
            continue
        ser = _extract_series(df_raw, c)
        score = _looks_like_direction(ser)
        if score > best_score:
            best_c, best_score = c, score
    return best_c if best_score >= 0.30 else None  # 30% 이상이면 방향으로 인정


def normalize_delay_table(df_raw: pd.DataFrame, date_label: str) -> pd.DataFrame:
    """
    임의 헤더 df_raw → 고정 스키마 [날짜, 노선, 방향, 첫차~09시, 09시~18시, 18시~막차]
    - rowspan/colspan 전개된 df_raw를 대상으로 동작
    - 헤더에 '방향'가 없어도 값 패턴으로 자동 매핑
    - 노선 셀 안에 방향이 붙어 있으면 분리
    """
    if df_raw is None or df_raw.empty:
        return pd.DataFrame([{
            "날짜": date_label, "노선": "", "방향": "", "첫차~09시": "", "09시~18시": "", "18시~막차": ""
        }], columns=TARGET_COLS)

    # 1) 시간대/노선/방향 헤더 매칭(있다면 우선)
    time_1_9 = time_9_18 = time_18_last = None
    explicit_line_cols, explicit_type_cols, ambig_cols = [], [], []

    for c in df_raw.columns:
        c_can = _canon(str(c))
        if time_1_9 is None and PAT_1_9.search(c_can):      time_1_9 = c;      continue
        if time_9_18 is None and PAT_9_18.search(c_can):    time_9_18 = c;     continue
        if time_18_last is None and PAT_18_LAST.search(c_can): time_18_last = c; continue

        if PAT_LINE_HDR.search(c_can):  explicit_line_cols.append(c);  continue
        if PAT_TYPE_HDR.search(c_can):  explicit_type_cols.append(c);  continue
        if PAT_AMBIG_HDR.search(c_can): ambig_cols.append(c);          continue

    chosen_line = explicit_line_cols[0] if explicit_line_cols else None
    chosen_type = explicit_type_cols[0] if explicit_type_cols else None

    # 2) '구분' 등 애매한 헤더는 값 패턴으로 분류
    for c in ambig_cols:
        ser = _extract_series(df_raw, c)
        if chosen_line is None and _looks_like_line(ser) >= 0.5:
            chosen_line = c
        elif chosen_type is None and _looks_like_direction(ser) >= 0.5:
            chosen_type = c

    # 3) 여전히 '방향'가 없으면: 모든 후보 중에서 방향값 비율이 가장 높은 컬럼을 선택
    exclude = set([x for x in [chosen_line, time_1_9, time_9_18, time_18_last] if x is not None])
    if chosen_type is None:
        chosen_type = _choose_direction_column(df_raw, exclude_cols=exclude)

    # 4) '노선'이 비어 있으면 보정(첫 컬럼 추정)
    if chosen_line is None and len(df_raw.columns) > 0:
        probe = df_raw.columns[0]
        if probe != chosen_type:
            chosen_line = probe

    # 5) 스키마 구성
    out = pd.DataFrame()
    out["날짜"] = [date_label] * len(df_raw)

    ser_line = _extract_series(df_raw, chosen_line) if (chosen_line is not None and chosen_line in df_raw.columns) else pd.Series([""] * len(df_raw))
    ser_type = _extract_series(df_raw, chosen_type) if (chosen_type is not None and chosen_type in df_raw.columns) else pd.Series([""] * len(df_raw))

    # 6) 노선 셀에 방향이 함께 들어간 경우 분리(방향가 비어있는 행만 채움)
    if ser_type.fillna("").eq("").any():
        new_line, new_type = [], []
        for ln, tp in zip(ser_line.fillna(""), ser_type.fillna("")):
            if tp:  # 이미 방향가 있으면 유지
                new_line.append(ln)
                new_type.append(tp)
            else:
                sp_ln, sp_tp = _split_line_and_dir(ln)
                new_line.append(sp_ln)
                new_type.append(sp_tp)
        ser_line = pd.Series(new_line)
        ser_type = pd.Series(new_type)

    out["노선"] = ser_line.fillna("").astype(str).str.strip()
    out["방향"] = ser_type.fillna("").astype(str).str.strip()

    # 7) 시간대 3칸(없으면 공란)
    out["첫차~09시"] = _extract_series(df_raw, time_1_9) if (time_1_9 is not None and time_1_9 in df_raw.columns) else ""
    out["09시~18시"] = _extract_series(df_raw, time_9_18) if (time_9_18 is not None and time_9_18 in df_raw.columns) else ""
    out["18시~막차"] = _extract_series(df_raw, time_18_last) if (time_18_last is not None and time_18_last in df_raw.columns) else ""

    # 완전 빈 DF 방어
    if out.empty:
        out = pd.DataFrame([{
            "날짜": date_label, "노선": "", "방향": "", "첫차~09시": "", "09시~18시": "", "18시~막차": ""
        }], columns=TARGET_COLS)

    # 컬럼 순서 고정
    out = out[TARGET_COLS]
    return out


# ==========================
# 메인 파이프라인
# ==========================
def scrape_delay_proofs(
    url: str = "http://www.seoulmetro.co.kr/kr/delayProofList.do?",
    output_csv: str = "delay_proof_all.csv",
    headless: bool = True
):
    driver = build_driver(headless=headless)
    all_dfs = []

    SELECT_LOC = (By.CSS_SELECTOR, "select#view_date")
    SEARCH_LOC = (By.CSS_SELECTOR, "a[href^='javascript:document.searchForm.submit']")

    try:
        driver.get(url)
        time.sleep(1.0)

        WebDriverWait(driver, 15).until(EC.presence_of_element_located(SELECT_LOC))
        WebDriverWait(driver, 15).until(EC.element_to_be_clickable(SEARCH_LOC))

        # 날짜 옵션 수집 (placeholder 제외)
        select_elem = driver.find_element(*SELECT_LOC)
        select_obj = Select(select_elem)
        option_meta = []
        for opt in select_obj.options:
            text = (opt.text or "").strip()
            value = (opt.get_attribute("value") or "").strip()
            if not value or "선택" in text:
                continue
            option_meta.append({"text": text, "value": value})

        if not option_meta:
            raise RuntimeError("select#view_date 에 유효한 날짜 option이 없습니다.")

        print(f"감지된 날짜 옵션 수: {len(option_meta)}")

        prev_sig = table_signature(safe_get_table(driver))

        for idx, meta in enumerate(option_meta, start=1):
            date_label = meta["text"] or meta["value"]

            # select 재조회(리렌더 대비)
            select_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located(SELECT_LOC))
            select_obj  = Select(select_elem)
            try:
                select_obj.select_by_value(meta["value"])
            except Exception:
                select_obj.select_by_visible_text(meta["text"])

            # 검색 클릭(JS)
            search_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(SEARCH_LOC))
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", search_btn)
            time.sleep(0.05)
            driver.execute_script("arguments[0].click();", search_btn)

            # ajax 갱신(or 타임아웃) 대기
            wait_for_update_or_timeout(driver, prev_signature=prev_sig, timeout=20)
            prev_sig = table_signature(safe_get_table(driver))
            time.sleep(0.15)

            # 파싱 → 정규화(고정 6컬럼, 방향=방향)
            table_el = safe_get_table(driver)
            raw_df = parse_table_to_df_rowspan(table_el)
            norm_df = normalize_delay_table(raw_df, date_label=date_label)
            all_dfs.append(norm_df)

            print(f"[진행] {idx}/{len(option_meta)}: {date_label} → 행수={len(norm_df)}")

        final_df = pd.concat(all_dfs, ignore_index=True)
        final_df = final_df[["날짜", "노선", "방향", "첫차~09시", "09시~18시", "18시~막차"]]
        final_df.to_csv('csv/'+output_csv, index=False, encoding="utf-8-sig")
        print(f"[완료] CSV 저장: {output_csv} (총 행수={len(final_df)})")

    finally:
        try:
            driver.quit()
        except Exception:
            pass


if __name__ == "__main__":
    # 디버깅 시 headless=False로 실제 DOM 변화를 확인하세요.
    scrape_delay_proofs(
        url="http://www.seoulmetro.co.kr/kr/delayProofList.do?",
        output_csv="delay_proof_all.csv",
        headless=True
    )


감지된 날짜 옵션 수: 31


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\3845288093.py:213: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\3845288093.py:271: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return (non_empty.str.contains(VAL_DIRECTION_RE) | non_empty.str.contains(r"(상행|하행)$")).mean()


[진행] 1/31: 금일 (2025-09-16) → 행수=23


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\3845288093.py:213: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\3845288093.py:271: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return (non_empty.str.contains(VAL_DIRECTION_RE) | non_empty.str.contains(r"(상행|하행)$")).mean()


[진행] 2/31: 1일전 (2025-09-15) → 행수=23


...

[진행] 29/31: 28일전 (2025-08-19) → 행수=23


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\3845288093.py:213: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\3845288093.py:271: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return (non_empty.str.contains(VAL_DIRECTION_RE) | non_empty.str.contains(r"(상행|하행)$")).mean()


[진행] 30/31: 29일전 (2025-08-18) → 행수=23


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\3845288093.py:213: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[~(df.applymap(lambda x: _clean_text(str(x))).eq("")).all(axis=1)].reset_index(drop=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22208\3845288093.py:271: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return (non_empty.str.contains(VAL_DIRECTION_RE) | non_empty.str.contains(r"(상행|하행)$")).mean()


[진행] 31/31: 30일전 (2025-08-17) → 행수=23
[완료] CSV 저장: delay_proof_all.csv (총 행수=713)
